In [4]:
import boto3
import awswrangler as wr
import os 
bucket = 'poc-2209-twayairport-dp'

s3 = boto3.client('s3')

In [10]:
key = 'train_data/coupon/pps_data/2018/08/coupon.parquet'

os.environ['snsARN'] = 'arn:aws:sns:ap-northeast-2:279545419827:tway_lambda_report'

In [3]:
52936704/1048576

50.484375

In [ ]:
186507264

In [17]:
10427854848/1048576

9944.77734375

In [16]:
import psutil
p = psutil.Process()
p.memory_info()
# 1,048,576

pmem(rss=10427854848, vms=13018349568, shared=74444800, text=2031616, lib=0, data=11508637696, dirty=0)

In [12]:
import boto3
import pandas as pd
import awswrangler as wr
import datetime 
import module_v1
from contextlib import suppress
print(key)

## Preprocessing 2

## Load coupon & booking Data
a, b, c, year, month, filename = key.split('/')
end_date = datetime.datetime.strptime(f'{year}-{int(month) + 1}','%Y-%m') - datetime.timedelta(days=1)
start_date = end_date - datetime.timedelta(days=365)
date_range = pd.date_range(start_date, end_date, freq= 'M')

coupon_list = []
booking_list = []

for idx, date in enumerate(date_range):
    if len(str(date.month)) == 1 :
        temp_month = '0'+str(date.month)
    else:
        temp_month = date.month        
    with suppress(Exception): coupon_list.append(wr.s3.read_parquet(f"s3://{bucket}/train_data/coupon/pps_data/{date.year}/{temp_month}/coupon_df.parquet"))
    with suppress(Exception): booking_list.append(wr.s3.read_parquet(f"s3://{bucket}/train_data/booking/pps_data/{date.year}/{temp_month}/booking_df.parquet"))
    
coupon_df = pd.concat(coupon_list)
booking_df = pd.concat(booking_list)
    
## Merge coupon_df & booking_df
orig_df = pd.merge(booking_df, coupon_df, on=list(set(list(booking_df)) & set(list(coupon_df))), how = 'right')
orig_df.dropna(axis = 0, inplace = True)

## Load flight Data
print(orig_df['flight_departure_date'].min(),orig_df['flight_departure_date'].max())
flight_range = pd.date_range(orig_df['flight_departure_date'].min(),orig_df['flight_departure_date'].max())
flight_df = pd.DataFrame()
for date in flight_range:
    if len(str(date.month)) == 1 :
        temp_month = '0'+str(date.month)
    else:
        temp_month = date.month
    if len(str(date.day)) == 1 :
        temp_day = '0'+str(date.day)
    else:
        temp_day = date.day
    with suppress(Exception): flight_df = pd.concat([flight_df, wr.s3.read_parquet(f"s3://{bucket}/train_data/flight/pps_data/{date.year}/{temp_month}/{temp_day}/flight_df.parquet")])

flight_raw = flight_df.shape
coupon_raw = coupon_df.shape
booking_raw = booking_df.shape

## Drop Duplicates
coupon_df.drop_duplicates(inplace = True)
booking_df.drop_duplicates(inplace = True)
flight_df.drop_duplicates(inplace = True)

## Merge coupon_df & booking_df & flight_df
orig_df = pd.merge(flight_df, orig_df, on = list(set(list(flight_df)) & set(list(orig_df))), how = 'right')
orig_df.reset_index(drop = True, inplace = True)

## Create Columns
total_df = module_v1.create_columns(orig_df, booking_df)
total_df.dropna(axis = 0, inplace = True)

## Save Data
wr.s3.to_parquet(df = total_df,
                 path=f"s3://{bucket}/train_data/{datetime.datetime.today().strftime('%Y-%m-%d %H')}/train_data.parquet")   

## Data Report
sns_arn = os.environ['snsARN']
snsclient = boto3.client('sns')
try:
    message = ""
    message += "\nLambda Data Report" + "\n\n"
    message += "############################################################################\n"
    message += "# Data PreProcessing & Train Dataset Save Done "+ "\n"
    message += "# Coupon Shape : " + str(coupon_raw) + " -> Drop Duplicates Shape : " + str(coupon_df.shape) + "\n"
    message += "# Booking Shape : " + str(booking_raw) + " -> Drop Duplicates Shape : " + str(booking_df.shape) + "\n"
    message += "# Flight Shape : " + str(flight_raw) + " -> Drop Duplicates Shape : " + str(flight_df.shape) + "\n\n"
    message += "# All Join Data Shape : " + str(orig_df) + " -> After PreProcessing 2 Shape : " + str(total_df.shape) + "\n"
    message += "############################################################################\n"

    snsclient.publish(
        TargetArn=sns_arn,
        Subject=f'Lambda Data Report',
        Message=message
    )
except Exception as e:
    print(e)

train_data/coupon/pps_data/2018/08/coupon.parquet
2017-08-01 00:00:00 2018-08-31 00:00:00


In [ ]:
import json
import boto3
import datetime
#import sagemaker

def lambda_handler(event, context):
    sm_boto3 = boto3.client("sagemaker")
    s3 = boto3.client('s3')
    project = 'train-from-boto3'
    source = 'source.tar.gz'

    # features 정의 필요
    # features = ['departure_hour', 'departure_minute', 'departure_time_format_min', 'sold_seats', 'total_sold_seats', 'remain_days']
    features = ['total_sold_seats', 'startseg_CJU', 'startseg_GMP', 'remain_seats', 'departure_hour', 'departure_minute', 'departure_time_format_min', 'remain_days', 
                'departureweekday_0', 'departureweekday_1', 'departureweekday_2', 'departureweekday_3', 'departureweekday_4', 'departureweekday_5', 'departureweekday_6', 
                'issueweekday_0', 'issueweekday_1', 'issueweekday_2', 'issueweekday_3', 'issueweekday_4', 'issueweekday_5', 'issueweekday_6']
    target = 'fare'
    training_image = '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3'
    role = 'arn:aws:iam::279545419827:role/service-role/AmazonSageMaker-ExecutionRole-20220901T101144'
    
    ## Load Train Data
    obj_list = s3.list_objects(Bucket = bucket, Prefix = 'train_data')
    obj_list = [i['Key'].split('/')[1] for i in obj_list['Contents'] if 'train_data.parquet' in i['Key']]   
    s3uri = f's3://poc-2209-twayairport-dp/train_data/{max(obj_list)}/train_data.parquet'
    
    ## Start Training Job    
    response = sm_boto3.create_training_job(TrainingJobName="sklearn-boto3-" + datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S"),
                                            HyperParameters={
                                                "sagemaker_program": "script.py",
                                                "features": ' '.join(features),
                                                "target": target,
                                                "sagemaker_submit_directory": f"s3://{bucket}/{project}/{source}",
                                            },
                                            AlgorithmSpecification={
                                                "TrainingImage": training_image,
                                                "TrainingInputMode": "File",
                                                "MetricDefinitions": [{"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"}],
                                            },
                                            RoleArn=role,
                                            InputDataConfig=[{"ChannelName": "total",
                                                              "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix",
                                                                                              "S3Uri": s3uri,
                                                                                              "S3DataDistributionType": "FullyReplicated",
                                                                                             }}}],
                                            OutputDataConfig={"S3OutputPath": f"s3://{bucket}/sagemaker-sklearn-artifact/"},
                                            ResourceConfig={"InstanceType": "ml.c5.xlarge", "InstanceCount": 1, "VolumeSizeInGB": 10},
                                            StoppingCondition={"MaxRuntimeInSeconds": 86400},
                                            EnableNetworkIsolation=False)  
    print(response)
    
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }
